# 1.Transformer介绍
Transformer是谷歌于2017年提出的一个用于解决序列生成任务的网络模型。自提出至今，Transformer的论文已经被引用七万多次，足以见其影响力之大。Transformer是第一个纯粹基于自注意力机制的序列模型，彻底抛弃了之前对RNN或者CNN的依赖。Transformer架构简洁，设计高度模块化，规模易拓展且利于并行计算，得益于这些优势，Transformer在NLP领域大放异彩，几乎所有NLP任务上的state-of-the-art模型都是基于Transformer架构，此外，随着VIT,SWIN Transformer的提出，Transformer也被应用到了计算机视觉领域，并且展示出了极大的潜力。

在本篇教程中，我将解释Transformer中的一些核心概念，并将手把手地教你实现一个Transformer

# 2.重要的核心概念
### 2.1 什么是Embedding
Embedding，也叫词嵌入技术，通过将语言中的单个字或者词映射为一个n维的数值向量，从而便于计算机进行计算，便于神经网络进行处理。

针对某种特定语言中字词的编码问题，很容易想到的一种编码方式就是one-hot编码。很显然，one-hot编码存在两个缺点，第一是one-hot的编码效率低下，大部分的码位都是0，只有一位是1，容易浪费计算机的存储空间，第二个更严重的缺点是，one-hot方式编码下，每一个编码与其它编码都是垂直正交的，而我们知道，字词之间实际上是存在相关性的，比如近义词之间的相关性就比不是近义词的两个词之间的相关性或者说相似性要高一些，而one-hot的编码并不能反映出这种相关性。

为了解决one-hot编码的缺陷，研究人员便发明了词嵌入技术，也就是Embedding。Embedding技术通过将词映射为一个n维的实向量，维度低于one-hot的编码，这样可以获得更高效，更紧凑的编码标示。此外，Embedding获得的向量可以反应词之间的相关性或者相似性，含义接近或者说相关的词，会被映射到空间中相临近的位置。
可以直接使用预训练好的Embedding，也可以自定义一个可训练的Embedding，在训练过程中同神经网络的其它部分一起更新。
### 2.2 什么是位置编码（Positional encoding）
使用RNN结构的模型处理序列数据时，要获得第k个输出，必须先获得前面所有的k-1个输出和隐状态, 第k-1个隐状态编码了前k-1个token的信息。Transformer为了能更好地进行并行计算，抛弃了顺序计算的RNN结构，通过自注意力机制对每个词向量进行重编码，使每个词向量都包含句子中的其它词的信息，但是自注意力机制并不能建模token在序列中的位置信息，比如对"我爱你"和"你爱我"进行自注意力机制后，编码后的词向量是完全相同的，而显然我们应该希望这两者编码后的结果是不同的。    
为了补充token在序列中的位置信息，Transformer引入了位置编码（positional encoding)。
Transformer论文中介绍的位置编码方法如下：
$PE(pos, 2i)=sin(pos/1000^{2i/dmodel})$    
$PE(pos, 2i+1)=cos(pos/1000^{2i/dmodel})$  

其中    
+ pos表示token在序列中的位置索引,对于语言来说，即表示这是第几个单词
+ i表示是词嵌入向量的第几个维度    

对于嵌入向量的偶数位，使用变周期的正弦函数来进行编码，对于嵌入向量的奇数位，使用变周期的余弦函数来进行编码。    
需要注意的是，使用变周期的三角函数来做位置编码，并不是唯一的方式，只是论文中恰好选用了这种方式。学习此处时，可不必拘泥于计算细节。
而更应理解使用位置编码的原因和一个好的位置编码函数应该具有什么特点。
一般来说，一个好的位置编码函数应该具有以下两个特点：
+ 数量级稳定且有界，不会过大或者过小
+ 应该是连续且平滑渐变的，能反映位置间的相对距离
+ 编码空间够大，能够避免编码碰撞

不难看出，正弦函数和余弦函数都满足前两个要求，为了满足第三个特性，Transformer中设计了变周期的三角函数和奇偶交替的方式，从而避免了编码重复

### 2.3 什么是自注意力机制
$Attention(Q, K, V) = softmax(QK^T / \sqrt d_k) * V$    

其中

+ Q 表示查询向量（Query），
+ K 表示键向量（Key），
+ V 表示值向量（Value），
+ softmax 表示softmax函数，
+ d_k 表示查询和键的维度。

上面就是大名鼎鼎的自注意力机制的计算公式。从形式上来看，就是两个矩阵乘的结果除以一个常数，然后经过softmax函数，再与另一个矩阵相乘得到最终的结果。    
在深入理解自注意力的含义之前，我们先复习一些关于向量的基础知识。
在介绍Embedding时，我们提到embedding之后的向量可以反应词之间的相关性。如何度量这种相关性呢? 在机器学习和自然语言处理等领域，我们常常使用向量之间的内积（或点积）来度量它们的相关度或相似度。这是因为内积在某种程度上可以捕捉到向量之间的几何关系和方向性。当两个向量的夹角 θ 接近于0度时，即它们趋向于同一方向，内积的结果将趋近于两个向量模的乘积，表示它们的相似度较高。而当夹角 θ 接近于90度时，即它们趋向于正交（垂直）的关系，内积的结果将趋近于0，表示它们的相似度较低。请先记住这个结论，<font color=##FF0000>两个向量的点积可以表示两个向量的相似度</font>    
下面我将举一个具体的例子来解释自注意力机制的原理
![embedding](pngs/embedding.png)
假设我们有这样一个句子：“我爱你”。每个字可以用一个向量表示，那么整个句子可以用一个（3，5）的矩阵来表示
![Q*K^T](pngs/mat_mul.png)
我们将这个矩阵与它的转置矩阵做转置乘法，可以得到一个（3，3）的矩阵。如上图所示。矩阵中的结果来自词向量间两两做内积，即整个矩阵描述了词和词之间的相关性。比如第一行的三个数字14.26， 9.2， 13.95，就分别表示了“我与我”，“我与爱”， “我与你”之间的相关性。<font color=##FF0000>注意，公式和实际的Transformer实现中，还需要给得到的这个矩阵除以$\sqrt {dmodel}$, 目的是防止点积后的结果过大导致梯度不稳定，此处演示简化了这一步，没有除以$\sqrt {dmodel}$</font> 

在每一行内使用softmax函数进行处理，就可以得到一个归一化之后的相似度矩阵。
![softmax_res](pngs/softmax_res.png)
知道了词与词之间的相关性之后，就可以对词向量进行加权融合了。  
![output](pngs/self_attention_output.png)
比如"我"，原本的词向量为(0.5, 0.1, 1, 2, 3)，和句子中所有其它词根据相似度矩阵中的权重进行融合后得到的新的词向量为(0.628, 0.2755, 1.429, 2, 2.781)。融合后，每个词向量不仅包含了这个词本身的含义，还编码了句子中其它字词的含义，再加上前面介绍的位置编码，就获得了句子中完整的上下文信息(context)。
### 2.4 什么是多头自注意力
上面我们介绍了自注意力的计算方式和它的含义，可以看到，自注意力没有任何参数，在embedding确定的情况下，自注意力的结果也是确定的。为了让自注意力机制能够有学习能力，我们可以为它加上参数，使用线性层对Q,K,V矩阵进行重新映射
即$Q=QW^Q$,$K=KW^K$,$V=VW^V$。
为了让自注意力机制的学习能力更强，Transformer中没有只使用一个线性层来投影，而是将Q,K,V都分成N份，每份都有自己的投影层，然后独立计算self attention,然后再把每一份的输出汇总在一起，这就是多头注意力的由来。在Transformer的论文中，设置N=8.具体实现时，通过reshape和transpose操作将形状如（batch,seq_length, dmodel）的矩阵调整为（batch, N, seq_length, dmodel//N），然后在最后两个维度上进行矩阵乘计算自注意力。使用这样的多头注意力而不是单个的注意力，可以提高模型的表达能力。
### 2.5 什么是Feed forward network
所谓的Feed forward network，其实就是一个两层的全连接网络，其中使用了relu作为激活函数，使用了dropout来缓解过拟合
### 2.6 关于网络结构的其它小细节
Transformer的网络中还有一些其它小细节需要注意

Transformer中实际上存在三种不同的多头自注意力，一种是编码器中的用于处理输入的自注意力，计算时输入的Q,K,V完全相同，都是上一层的输出，另一种是解码器中用于处理shifted right序列的masked自注意力，计算时输入的QKV也完全相同，第三种是解码器中用于融合编码器的输出的自注意力层，称为交叉自注意力层，其中的Q来自解码器的上一层输出，K和V都来自编码器的输出。


在处理语言或者其它序列数据时，由于每条序列的长度各异，在打包成batch时，必须对其中较短的序列进行补齐,也就是padding,补齐时会选择一个特定的值作为padding值，比如0，而在神经网络计算时，我们希望网络能够忽略这些padding值对应的信息，不被这些占位符影响。

此外，Transformer中还存在另外一种mask，在使用解码器来做自回归预测时，预测第k个输出时，只能看到前面k-1个已经预测的输出。而在训练时，由于我们使用了shifted right,相当于网络可以看到整个句子，为了避免网络学会作弊，需要进行掩码处理，即句子中的第i个字词，在进行自注意力融合时，只能和自己所处位置之前的其它字符进行融合，而不能和自己之后的字符进行融合。

推理和训练时候的区别
训练时候为了避免网络的预测值出现过大的偏差，影响后续token的预测，故将ground truth右移得到shifted right，然后将shifted right作为解码器的输入，相当于直接使用正确的标签来进行自回归，这称为teacher forcing. 在测试时，没有正确的标签了，所以只能使用网络预测的前k-1个字符去预测第k个字符，这才是标准的自回归操作，特别的，在预测第一个字符时，由于前面还没有产生预测值，所以需要放置一个起始字符来作为解码器的输入。

# 3. 从零构建Transformer
![Transformer模型结构图](pngs/architecture.png)

### 3.1 环境配置

In [ ]:
!pip install datasets
!pip install tokenizers
!pip install spacy
!pip3 install torch torchvision torchaudio torchtext --index-url https://download.pytorch.org/whl/cu117
!python -m spacy download en_core_web_sm
!python -m spacy download pt_core_news_sm

In [ ]:
# 导入必要的模块
import numpy as np
from torch import nn
import torch
import math
from collections import Counter, OrderedDict
from torch.utils.tensorboard import SummaryWriter
from datasets import load_dataset
from torchtext.data.utils import get_tokenizer
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch.nn.functional import log_softmax
from torch.optim.lr_scheduler import LambdaLR
from torchtext.vocab import vocab
from functools import partial
from torch.nn.utils.rnn import pad_sequence

### 3.2 Embedding层的实现
embedding层的作用是将由字典中的位置索引表示的词映射成一个n维的向量，使用这个向量来表示单词。    
embedding层有两个参数：
+ vocal_size表示词典的长度
+ d_model表示向量的维度，在Transformer论文中，d_model被设置为512

In [ ]:
# LayerNorm层的实现，也可以直接使用nn.LayerNorm
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."

    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


# 词嵌入层
# 将token在词典中的id映射为一个d_model维的向量
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, x):
        """
        :param x: tokenlized sequence
        :return:
        """
        # 乘以一个较大的系数，放大词嵌入向量，
        # 希望与位置编码向量相加后，词嵌入向量本身的影响更大
        return self.embed(x) * math.sqrt(self.d_model)

### 3.3 位置编码层的实现


In [ ]:
# 位置编码。这段代码的实现是抄的
# 预先计算好所有可能的位置编码，然后直接查表就能得到
# 注意维度顺序
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 1000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        pe = pe.permute(1, 0, 2)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # X的形状为 (batch_size, seq_length, d_model)
        x = x + self.pe[:, :x.size(1), :].requires_grad_(False)
        return self.dropout(x)

### 3.4 多头注意力Multi head attention层的实现

In [ ]:
# 多头自注意力
# 这段代码的实现也是抄的现成的。奇文共欣赏，疑义相与析
class MultiHeadAttention(nn.Module):
    def __init__(self, head_number, d_model):
        """
        :param head_number: 自注意力头的数量
        :param d_model: 隐藏层的维度
        """
        super().__init__()
        self.h = head_number
        self.d_model = d_model
        self.dk = d_model // head_number
        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.output = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(-1)
        self.dropout = nn.Dropout(0.1)

    def head_split(self, tensor, batch_size):
        # 将(batch_size, seq_len, d_model) reshape成 (batch_size, seq_len, h, d_model//h)
        # 然后再转置第1和第2个维度，变成(batch_size, h, seq_len, d_model/h)
        return tensor.view(batch_size, -1, self.h, self.dk).transpose(1, 2)

    def head_concat(self, similarity, batch_szie):
        # 恢复计算注意力之前的形状
        return similarity.transpose(1, 2).contiguous() \
            .view(batch_szie, -1, self.d_model)

    def cal_attention(self, q, k, v, mask=None):
        """
        论文中的公式 Attention(K,Q,V) = softmax(Q@(K^T)/dk**0.5)@V
        ^T 表示矩阵转置
        @ 表示矩阵乘法
        """
        similarity = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.dk)
        if mask is not None:
            mask = mask.unsqueeze(1)
            # 将mask为0的位置填充为绝对值非常大的负数
            # 这样经过softmax后，其对应的权重就会非常接近0, 从而起到掩码的效果
            similarity = similarity.masked_fill(mask == 0, -1e9)
        similarity = self.softmax(similarity)
        similarity = self.dropout(similarity)

        output = torch.matmul(similarity, v)
        return output

    def forward(self, q, k, v, mask=None):
        """
        q,k,v即自注意力公式中的Q,K,V，mask表示掩码
        """
        batch_size, seq_length, d = q.size()
        q = self.q_linear(q)
        k = self.k_linear(k)
        v = self.v_linear(v)
        # 分成多个头
        q = self.head_split(q, batch_size)
        k = self.head_split(k, batch_size)
        v = self.head_split(v, batch_size)
        similarity = self.cal_attention(q, k, v, mask)
        # 合并多个头的结果
        similarity = self.head_concat(similarity, batch_size)

        # 再使用一个线性层， 投影一次
        output = self.output(similarity)
        return output

### 3.5 前馈神经网络的实现
不要被名字唬到，前馈神经网络就是全连接神经网络

In [ ]:
# 论文中的前馈神经网络。实际上就是一个两层的全连接，中间加上个relu和dropout
class FeedForwardNetwork(nn.Module):
    def __init__(self, d_model, dff, dropout=None):
        super().__init__()
        self.fc1 = nn.Linear(d_model, dff)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(dff, d_model)
        if dropout is not None:
            self.dropout = nn.Dropout(0.1)
        else:
            self.dropout = None

    def forward(self, x):
        """
        :param x: 来自多头自注意力层的输出
        :return:
        """
        x = self.fc1(x)
        x = self.relu(x)
        if self.dropout is not None:
            x = self.dropout(x)
        output = self.fc2(x)
        return output

### 3.6 编码器层的实现

In [ ]:
# 编码器层。
# 每个编码器层由两个sublayer组成，即一个多头注意力层和一个前馈网络
class EncoderLayer(nn.Module):
    def __init__(self, head_number, d_model, d_ff, dropout=0.1):
        super().__init__()

        # mha
        self.mha = MultiHeadAttention(head_number, d_model)
        self.norm1 = LayerNorm(d_model)

        # mlp
        self.mlp = FeedForwardNetwork(d_model, d_ff)
        self.norm2 = LayerNorm(d_model)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, x, mask=None):
        x2 = self.norm1(x)
        
        y = self.dropout1(self.mha(x2, x2, x2, mask))
        # 注意残差连接是和norm之前的输入相加，norm之后的不在一个数量级
        y = y + x
        
        y2 = self.norm2(y)
        y2 = self.dropout2(self.mlp(y2))
        y2 = y + y2

        return y2

### 3.7 编码器的实现

In [ ]:
# 编码器部分
# 编码器就是N个编码器层堆叠起来。论文中为6个编码器层
class Encoder(nn.Module):
    def __init__(self, stack=6, multi_head=8, d_model=512, d_ff=2048):
        """
        :param stack: 堆叠多少个编码器层
        :param multi_head: 多头注意力头的数量
        :param d_model: 隐藏层的维度
        """
        super().__init__()
        self.encoder_stack = []
        for i in range(stack):
            encoder_layer = EncoderLayer(multi_head, d_model, d_ff)
            self.encoder_stack.append(encoder_layer)
        self.encoder = nn.ModuleList(self.encoder_stack)
        self.norm = LayerNorm(d_model)

    def forward(self, x, mask=None):
        for encoder_layer in self.encoder:
            x = encoder_layer(x, mask)
        x = self.norm(x)
        return x

### 3.8 解码器层的实现

In [ ]:
# 解码器层
# 一个解码器层由三个sublayer组成，即一个masked自注意力层，一个cross自注意力层和一个前馈网络层
# cross自注意力层意思是q,k,v分别来自编码器和解码器
class DecoderLayer(nn.Module):
    def __init__(self, head_number, d_model, d_ff, dropout=0.1):
        super().__init__()
        # shifted right self attention layer
        self.mha1 = MultiHeadAttention(head_number, d_model)

        # cross attention
        self.mha2 = MultiHeadAttention(head_number, d_model)

        self.norm1 = LayerNorm(d_model)
        self.norm2 = LayerNorm(d_model)
        self.norm3 = LayerNorm(d_model)

        self.mlp = FeedForwardNetwork(d_model, d_ff, 0.1)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, q, k, v, src_mask, tgt_mask):
        # 注意第一个注意力层的qkv都是同一个
        x2 = self.norm1(q)
        y = self.mha1(x2, x2, x2, tgt_mask)
        y = self.dropout1(y)
        
        # 注意残差连接是和norm之前的输入相加，norm之后的不在一个数量级
        y = y + q
        
        y2 = self.norm2(y)
        
        # 第二个自注意力层的k和v是encoder的输出
        y2 = self.mha2(y2, k, v, src_mask)
        y2 = self.dropout2(y2)
        y2 = y + y2
        
        y3 = self.norm3(y2)
        y3 = self.mlp(y3)
        y3 = self.dropout3(y3)
        y3 = y2 + y3

        return y3

### 3.9 解码器的实现

In [ ]:
# 解码器
# 解码器就是N个解码器层堆叠起来
class Decoder(nn.Module):
    def __init__(self, stack=6, head_number=8, d_model=512, d_ff=2048):
        super().__init__()
        self.decoder_stack = []
        for i in range(stack):
            self.decoder_stack.append(DecoderLayer(head_number, d_model, d_ff))
        self.decoder_stack = nn.ModuleList(self.decoder_stack)
        self.norm = LayerNorm(d_model)

    def forward(self, x, output_from_encoder, src_mask, tgt_mask):
        for decoder_layer in self.decoder_stack:
            x = decoder_layer(x, output_from_encoder, output_from_encoder, src_mask, tgt_mask)
        x = self.norm(x)
        return x

### 3.10 最终的变形金刚(Transformer)
将编码器和解码器组合起来，再加上输入和最终的输出层，就可以得到一个完整的Transformer模型了

In [ ]:
# 最终的变形金刚
# 汽车人出发！！！！！！！！！！！！！！！！
class TransformerModel(nn.Module):
    def __init__(self,
                 src_voc_size,
                 target_voc_size,
                 stack_number=6,
                 d_model=512,
                 h=8,
                 d_ff=2048):
        super().__init__()
        self.input_embedding_layer = Embedder(src_voc_size, d_model)
        self.input_pe = PositionalEncoding(d_model)

        self.output_embedding_layer = Embedder(target_voc_size, d_model)
        self.output_pe = PositionalEncoding(d_model)

        self.encoder = Encoder(stack_number, h, d_model, d_ff)
        self.decoder = Decoder(stack_number, h, d_model, d_ff)
        self.final_output = nn.Linear(d_model, target_voc_size)

    def encode(self, src, src_mask):
        x = self.input_embedding_layer(src)
        x = self.input_pe(x)
        output_from_encoder = self.encoder(x, src_mask)
        return output_from_encoder

    def decode(self, output_from_encoder, shifted_right, src_mask, tgt_mask):
        shifted_right = self.output_embedding_layer(shifted_right)
        shifted_right = self.output_pe(shifted_right)

        decoder_output = self.decoder(shifted_right, output_from_encoder, src_mask, tgt_mask)

        output = self.final_output(decoder_output)
        return output

    def forward(self, x, shifted_right, src_mask, tgt_mask):
        x = self.input_embedding_layer(x)
        x = self.input_pe(x)

        output_from_encoder = self.encoder(x, src_mask)

        shifted_right = self.output_embedding_layer(shifted_right)
        shifted_right = self.output_pe(shifted_right)

        decoder_output = self.decoder(shifted_right, output_from_encoder, src_mask, tgt_mask)

        output = log_softmax(self.final_output(decoder_output), dim=-1)

        return output


def build_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    model = TransformerModel(src_vocab, tgt_vocab, N, d_model, h, d_ff)
    # 权重初始化
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    return model



# 简单测试。
# 验证网络的结构的连通性和输入输出的维度
def basic_test():
    a = np.random.randint(0, 100, size=(1, 10)).astype(np.int32)
    b = torch.from_numpy(a)

    a2 = np.random.randint(0, 100, size=(1, 12)).astype(np.int32)
    b2 = torch.from_numpy(a2)
    model = build_model(100, 120)
    print(model)
    output = model(b, b2, None, None)
    print(output.shape)
basic_test()

# 4. 训练Transformer用作机器翻译

## 4.1 数据集加载
ted_hrlr/pt_to_en是一个葡萄牙语与英语的文本数据集，其中包含了大约五万组<葡萄牙语-英语>的句子对。方便起见，使用Huggingface的dataset来读取这个数据集,如果下载失败，请尝试使用代理

In [ ]:
from datasets import load_dataset
dataset = load_dataset('ted_hrlr', name='pt_to_en')

In [ ]:
# 简单check
train_data = dataset['train']['translation']
print(type(train_data))
print("训练集数量为:{}\n".format(len(train_data)))
for data in train_data[:5]:
    en_seq = data['en']
    pt_seq = data['pt']
    print('英语句子为:\n{}'.format(en_seq))
    print('葡萄牙语句子为:\n{}'.format(pt_seq))
    print('\n')

## 4.2 Tokenize以及字典
句子就是一个由一组词语按一定顺序组成的，在计算机中，词语和句子都可以由字符串来进行表示。而神经网络是无法直接处理字符串的，因此，有必要对单词进行编码，使用数字来表示单词。在数字和单词之间建立起映射关系，就是通过tokenize和建立字典来完成的。
Tokenize就是将一个句子转化为一组基本的token，token表示最小的语义单位。常用的tokenize的方法根据粒度的大小可以分为单词法，子词法和字符法。
将句子中所有的token提取出来之后，就可以为token建立字典，将每个token与一个整数标示的ID一一对应起来，这样就可以获得token的数字化表示，也就可以得到一个句子的数字化表示

In [ ]:
def build_vocab(seqs, tokenizer, k=None):
    counter = Counter()
    for seq in seqs:
        if k is not None:
            seq = seq[k]
        token = tokenizer(seq)
        counter.update(token)
    sorted_by_freq_tuples = sorted(counter.items(),
                                   key=lambda x: x[1],
                                   reverse=True)
    ordered_dict = OrderedDict(sorted_by_freq_tuples)
    # 指定特殊字符，特殊字符会被放在字典的起始未知，比如'<pad>'的索引就是0
    voc = vocab(ordered_dict, specials=['<pad>', '<unk>', '<bos>', '<eos>'])
    return voc

## 4.3 构建数据集和DataLoader

In [ ]:
def generate_batch(data_batch):
    pt_batch, en_input_batch, en_label_batch = [], [], []
    for pt, eni, enl in data_batch:
        pt_batch.append(pt)
        en_input_batch.append(eni)
        en_label_batch.append(enl)

    pt_batch = pad_sequence(pt_batch, padding_value=0, batch_first=True)
    # 使用0来补齐
    en_input_batch = pad_sequence(en_input_batch, padding_value=0, batch_first=True)
    en_label_batch = pad_sequence(en_label_batch, padding_value=0, batch_first=True)

    return pt_batch, en_input_batch, en_label_batch


def seq_to_index(seq, tokenizer, voc):
    # 添加起始字符和结束字符
    indexs = [voc['<bos>']] + [voc[token] for token in tokenizer(seq)] + [voc['<eos>']]
    return torch.tensor(indexs, dtype=torch.int64)


class TranslationDataset(Dataset):
    def __init__(self, dataset, en_voc, pt_voc, en_tokenizer, pt_tokenizer):
        self.dataset = dataset
        self.en_tokenizer = en_tokenizer
        self.pt_tokenizer = pt_tokenizer
        self.en_voc = en_voc
        self.pt_voc = pt_voc

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, item):
        en_seq, pt_seq = self.dataset[item]['en'], self.dataset[item]['pt']

        # seq to token list. start and end token added
        en_tensor = seq_to_index(en_seq, self.en_tokenizer, self.en_voc)
        pt_tensor = seq_to_index(pt_seq, self.pt_tokenizer, self.pt_voc)

        ground_truth = en_tensor[1:]  # drop the start token
        shifted_right = en_tensor[:-1]  # drop the end token
        return pt_tensor, shifted_right, ground_truth

    
def build_dataloader(batch_size, cache_dir=None):
    # 读取《葡萄牙语-英语》翻译数据集
    dataset = load_dataset('ted_hrlr', name='pt_to_en', cache_dir=cache_dir)

    # 获取训练集切分和验证集切分
    train_data = dataset['train']['translation']
    val_data = dataset['validation']['translation']

    # 分词器
    en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
    pt_tokenizer = get_tokenizer('spacy', language='pt_core_news_sm')

    # 构建字典索引
    en_voc = build_vocab(train_data+val_data, en_tokenizer, k='en')
    pt_voc = build_vocab(train_data+val_data, pt_tokenizer, k='pt')

    # 创建dataloader
    train_ds = TranslationDataset(train_data, en_voc, pt_voc, en_tokenizer, pt_tokenizer)
    val_ds = TranslationDataset(val_data, en_voc, pt_voc, en_tokenizer, pt_tokenizer)
    train_iter = DataLoader(train_ds, batch_size=batch_size,
                            shuffle=True, collate_fn=generate_batch)
    # 将验证集的batch_size设置为1
    valid_iter = DataLoader(val_ds, batch_size=1,
                            shuffle=False, collate_fn=generate_batch)
    return train_iter, valid_iter, len(en_voc), len(pt_voc), en_voc


def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0


# 创建mask
def create_mask(src, tgt, pad=0):
    src_mask = (src != pad).unsqueeze(-2)
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
        tgt_mask.data
    )
    return src_mask, tgt_mask

## 4.5 训练
我们将在这个机器翻译数据集上训练60个epoch

In [ ]:
GLOBAL_STEP = 1
device='cuda'


def train_one_step(model, loss_fn, batch_data, 
                   optimizer, lr_scheduler=None, 
                   log_writter=None,
                   global_step=None):
    # 切换到训练模式
    model.train()
    
    # x1为源语言序列，x2为shifted right,label表示目标语言序列，也就是标签
    x1, x2, label = batch_data

    # 创建掩码
    # src掩码的作用是为了忽略padding的字符
    # tgt掩码的作用是自回归时候遮蔽当前位置之后的字符
    src_mask, tgt_mask = create_mask(x1, x2)

    x1 = x1.to(device)
    x2 = x2.to(device)
    label = label.to(device)
    src_mask = src_mask.to(device)
    tgt_mask = tgt_mask.to(device)
    
    # 前向获得输出
    pred = model(x1, x2, src_mask, tgt_mask)

    label = label.to(torch.long)
    pred = pred.contiguous().view(-1, pred.size(-1))
    label = label.contiguous().view(-1, )
    
    # 计算损失
    loss = loss_fn(pred, label)
    
    # 更新权重
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if lr_scheduler is not None:
        lr_scheduler.step()
    if log_writter is not None:
        log_writter.add_scalar('train_loss', loss, global_step)
    return loss


def rate(step, model_size=512, warmup=4000):
    # 学习率调度
    if step == 0:
        step = 1
    return (
            model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )


def train():
    # 超参数设置
    batch_size = 48
    epochs = 60
    checkpoint_interval = 1

    # 创建数据输入管道
    training_data, val_data, tgt_voc_size, src_voc_size, tgt_voc = build_dataloader(batch_size)

    # 构建模型
    model = build_model(src_voc_size, tgt_voc_size, N=6, d_ff=1024)
    model.to(device)

    # 优化器，注意此处的lr设置比较大，作为一个基础学习率，经过调度器缩放后，数量级会减小
    optimizer = torch.optim.Adam(
        model.parameters(), lr=1, betas=(0.9, 0.98), eps=1e-9
    )

    lr_scheduler = LambdaLR(
        optimizer=optimizer, lr_lambda=lambda step: rate(step)
    )

    # 交叉熵损失函数，使用label smoothing， 忽略标签中的0, 0表示补齐占位符
    loss_fn = partial(F.cross_entropy, ignore_index=0, label_smoothing=0.1)

#     logger = SummaryWriter('./logs/')
    logger = None
    # 训练Loop
    global GLOBAL_STEP
    for epoch in range(1, epochs + 1):
        for step, batch_data in enumerate(training_data):
            # 在一个batch上训练
            loss = train_one_step(model, loss_fn, batch_data, optimizer, lr_scheduler, log_writter=logger,
                                  global_step=GLOBAL_STEP)
            GLOBAL_STEP += 1
            if step % 200 == 0:
                print("epoch {}/{}  step:{}  loss:{:.3f}".format(epoch, epochs, step, loss))
        if epoch % checkpoint_interval == 0:
            # 周期性保存最新的模型
            torch.save(model.state_dict(), 'latest_ckpt.pth')
    
    if logger is not None:
        logger.flush()
        logger.close()
    torch.save(model.state_dict(), 'transformer.pth')


train()

## 5. 测试
注意在训练时候，通过舍弃掉ground truth序列的第一个字符得到shifted right，将得到的序列作为解码器的输入。    
而在测试时候，是没有ground truth序列的，所以需要一个单词一个单词地进行生成，生成第k个单词时，总是将前面k-1个已经生成的单词作为输入，这就是自回归的含义

In [ ]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, ys, src_mask, subsequent_mask(ys.size(1)).type_as(src.data)
        )[0]
        prob = F.softmax(out, dim=-1)
        # 每次取预测的最后一个字符
        next_word = torch.argmax(prob, dim=-1)[-1].data
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys


def token2text(tokens, voc_itos, eos=None):
    res = []
    for i, token in enumerate(tokens):
        c = voc_itos[token]
        if eos is not None:
            if c == eos:
                break
        res.append(c)
    return ' '.join(res)


def test():
    device = 'cuda'
    # dataloader
    batch_size = 1
    training_data, val_data, tgt_voc_size, src_voc_size, tgt_voc = build_dataloader(batch_size)
    # 构建模型
    model = build_model(src_voc_size, tgt_voc_size, N=6, d_ff=1024)
    # 加载训练好的模型
    model.load_state_dict(torch.load('transformer.pth'))
    model.to(device)
    
    # 切换到测试模式
    model.eval()
    max_len = 128
    itos = tgt_voc.get_itos()
    for _ in range(3):
        val_data = iter(val_data)
        batch_data = next(val_data)
        x1, x2, label = batch_data
        src_mask, tgt_mask = create_mask(x1, x2)
        x1 = x1.to(device)
        src_mask = src_mask.to(device)
        
        # 贪心解码
        pred_indexs = greedy_decode(model, x1, src_mask, max_len, 2)[0]
        
        # 将token ID转换成对应的字符
        pred_text = token2text(pred_indexs, itos, eos='<eos>')
        label = token2text(label[0], itos, eos='<eos>')
        print("实际的英语句子为:")
        print(label)
        print("翻译得到的句子为:")
        print(pred_text)

test()

# 6. 总结
至此，你已经成功实现并训练了一个你自己的Transformer模型。
如果你想继续学习关于Transformer的知识，或者想了解Transformer及其变体在视觉领域中的应用，下面是一些可供参考的资料:

[Annotated Transformer 哈佛大学的Transformer实现  (本文中部分代码借鉴于此)](http://nlp.seas.harvard.edu/annotated-transformer/)

[Transformer论文原文](https://arxiv.org/abs/1706.03762)

[李沐: Transformer论文精读](https://www.bilibili.com/video/BV1pu411o7BE/?spm_id_from=333.999.0.0&vd_source=7ba4ab07bcd248758aff19a21fc5010b)

[Vision Transformer论文原文](https://arxiv.org/abs/2010.11929)

[VIT论文精读](https://www.bilibili.com/video/BV15P4y137jb/?spm_id_from=333.999.0.0&vd_source=7ba4ab07bcd248758aff19a21fc5010b)

[Swin Transformer论文原文](https://arxiv.org/pdf/2103.14030.pdf)

[Swin Transformer论文精读](https://www.bilibili.com/video/BV13L4y1475U/?spm_id_from=333.999.0.0&vd_source=7ba4ab07bcd248758aff19a21fc5010b)